<a href="https://colab.research.google.com/github/mardhiahnajwa/assignments-dliva/blob/main/Assessment1_DeepLearning_MardhiahNajwa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nur Mardhiah Najwa Bt Mohd Nasri | 1815470 | IIUM**

## Assessment 1: Deep Learning

1) Answer all questions.

2) This assessment is open-book. You are allowed to refer to any references including online materials, books, notes, codes, github links, etc.

3) Copy this notebook to your google drive (click **FILE** > **save a copy in Drive**)

4) Upload the answer notebook to your github. 

5) Submit the assessment by sharing the link to your answer notebook. 





**QUESTION 1** 

One day while wandering around a clothing store at KL East Mall, you stumbled upon a person who is choosing a dress for Hari Raya. It turns out that the person is visually impaired and had a hard time distinguishing between an abaya and a kebaya. To help people with the similar situation, you then decided to develop an AI system to identify the type of clothes using a Convolutional Neural Networks (ConvNet). In order to train the network, you decide to use the Fashion MNIST dataset which is freely available on Pytorch.


a) Given the problem, what is the most appropriate loss function to use? Justify your answer. **[5 marks]**

**ANSWER**

<span style="color:blue">
    Loss function acts as an indicator to see if classification has a good data-point, so if there's a large loss function means that the model does not accurately predict the data.
    The most appropriate loss function is Cross Entropy Loss (CEL). When CEL is high, it means that the predicted data is not the same as actual data. Cross Entropy also manages to get the data-point from local minimum into global minimum, which is the ideal data-point for a good classification. In this case, to classify clothes in Fashion MNIST, the more the loss, the lower the performance of the model. </span>

b) Create and train a ConvNet corresponding to the following CNN architecture (with a modification of the final layer to address the number of classes). Please include **[10 marks]**:

    1) The dataloader to load the train and test datasets.

    2) The model definition (either using sequential method OR pytorch class method).

    3) Define your training loop.

    4) Output the mean accuracy for the whole testing dataset.

    



![pic](https://raw.githubusercontent.com/CUTe-EmbeddedAI/images/main/images/LeNet.png)

In [44]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import glob
import numpy
import random

from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchsummary import summary

In [45]:
# 1) The dataloader to load the train and test datasets.


transform = transforms.Compose(
     [transforms.ToTensor(),
      transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


 # batch_size
batch_size = 32

# datasets
trainset = torchvision.datasets.FashionMNIST('./data',
     download=True,
     train=True,
     transform=transform)
testset = torchvision.datasets.FashionMNIST('./data',
     download=True,
     train=False,
     transform=transform)

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
         'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

train_data_size = len(trainloader.dataset)
test_data_size = len(testloader.dataset)

print(train_data_size)
print(test_data_size)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

60000
10000


In [53]:
# 2) The model definition (either using sequential method OR pytorch class method.

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 10)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flatten(x)#x.view(-1, 16*5*5)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [54]:
# 3) Define your training loop.

model = ConvNet()

# LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# move the model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

ConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
  (relu): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
)

In [48]:
import time # to calculate training time

def train_and_validate(model, loss_criterion, optimizer, epochs):

    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in enumerate(trainloader):

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)

            loss = loss_criterion(outputs, labels)

            loss.backward()

            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            train_acc += acc.item() * inputs.size(0)

        # Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in enumerate(testloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)

                #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
            
        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size

        # Find average training loss and training accuracy
        avg_test_loss = valid_loss/test_data_size 
        avg_test_acc = valid_acc/test_data_size

        history.append([avg_train_loss, avg_test_loss, avg_train_acc, avg_test_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_test_loss, avg_test_acc*100, epoch_end-epoch_start))
        
        # Save if the model has best accuracy till now
        torch.save(model, 'cnn_fmnist_'+str(epoch)+'.pt')
            
    return model, history

In [55]:
# Train the model for 10 epochs

num_epochs = 10
trained_model, history = train_and_validate(model, criterion, optimizer, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.8879, Accuracy: 67.9167%, 
		Validation : Loss : 0.5547, Accuracy: 79.7000%, Time: 18.3424s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.4772, Accuracy: 82.3250%, 
		Validation : Loss : 0.4504, Accuracy: 83.5300%, Time: 21.3996s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.4080, Accuracy: 84.9467%, 
		Validation : Loss : 0.4357, Accuracy: 84.4300%, Time: 19.9058s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.3705, Accuracy: 86.4833%, 
		Validation : Loss : 0.3838, Accuracy: 86.0300%, Time: 18.6631s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.3481, Accuracy: 87.2350%, 
		Validation : Loss : 0.3701, Accuracy: 86.4100%, Time: 18.6677s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.3287, Accuracy: 88.0450%, 
		Validation : Loss : 0.3499, Accuracy: 87.0300%, Time: 18.8347s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.3136, Accuracy: 88.5983%, 
		Validation : Loss : 0.3483, Accuracy: 87.4100%, Time: 18.6512s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.3017, Accura

In [56]:
# 4) Output the mean accuracy for the whole testing dataset.

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')


Accuracy of the network on the 10000 test images: 88 %


c) Replace your defined CNN in b) with a pre-trained model. Then, proceed with a transfer learning and finetune the model for the Fashion MNIST dataset. **[10 marks]**

In [10]:
# c) pretrained model: googlenet

model_pt = models.googlenet(pretrained=True)
num_ftrs = model_pt.fc.in_features
model_pt.fc = nn.Linear(num_ftrs, 10)

model_pt = model_pt.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_pt.parameters(), lr=0.001, momentum=0.9)

model_pt

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [11]:
num_epochs = 10
trained_model, history = train_and_validate(model_pt, criterion, optimizer_ft, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.6631, Accuracy: 77.3833%, 
		Validation : Loss : 0.3534, Accuracy: 87.4600%, Time: 75.1238s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.3754, Accuracy: 86.6350%, 
		Validation : Loss : 0.3123, Accuracy: 88.9200%, Time: 72.9571s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.3108, Accuracy: 88.9900%, 
		Validation : Loss : 0.2793, Accuracy: 89.8100%, Time: 73.8943s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.2728, Accuracy: 90.1800%, 
		Validation : Loss : 0.2627, Accuracy: 90.4700%, Time: 72.6872s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.2487, Accuracy: 91.1017%, 
		Validation : Loss : 0.2556, Accuracy: 90.8900%, Time: 72.2377s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.2259, Accuracy: 91.8833%, 
		Validation : Loss : 0.2610, Accuracy: 90.4200%, Time: 70.9150s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.2092, Accuracy: 92.5783%, 
		Validation : Loss : 0.2393, Accuracy: 91.5500%, Time: 71.2331s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.1916, Accura

d) Using model-centric methods, propose two (2) strategies that can be used to increase the accuracy of the model on the testing dataset. **[5 marks]**


<span style="color:blue">
    The model-centric techniques that I propose is to increase the dropout layer. Other than that, alter the batch normalization mean and standard deviation to fit the weight that is in the hidden layer. The learning rate can also be altered to fit the training model.</span>

e) Next, implement the two proposed model-centric techniques for the same problem as in the previous question. **[15 marks]**

In [13]:
# e) two model centric techniques
model_pt = models.googlenet(pretrained=True)
num_ftrs = model_pt.fc.in_features
model_pt.fc = nn.Linear(num_ftrs, 10)
model_pt.dropout = nn.Dropout(p=0.25)

model_pt = model_pt.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_pt.parameters(), lr=0.005, momentum=0.9)

# train again
num_epochs = 10
trained_model, history = train_and_validate(model_pt, criterion, optimizer_ft, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.5258, Accuracy: 82.1150%, 
		Validation : Loss : 0.3064, Accuracy: 89.2200%, Time: 76.3452s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.3109, Accuracy: 89.1967%, 
		Validation : Loss : 0.2651, Accuracy: 90.6100%, Time: 76.9305s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.2634, Accuracy: 90.6750%, 
		Validation : Loss : 0.2649, Accuracy: 90.4400%, Time: 76.7725s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.2300, Accuracy: 91.8800%, 
		Validation : Loss : 0.2462, Accuracy: 90.8600%, Time: 76.5182s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.2055, Accuracy: 92.7833%, 
		Validation : Loss : 0.2358, Accuracy: 91.8100%, Time: 74.1890s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.1898, Accuracy: 93.1850%, 
		Validation : Loss : 0.2415, Accuracy: 91.4100%, Time: 75.1552s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.1721, Accuracy: 93.8733%, 
		Validation : Loss : 0.2291, Accuracy: 91.8600%, Time: 75.9196s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.1584, Accura

f) Do you see any accuracy improvement? Whether it is a "yes" or "no", discuss the possible reasons contributing to the accuracy improvement/ unimprovement. **[5 marks]**

<span style="color:blue">
    Yes, there is an improvement. This is due to having a more dropout makes the model to drop a lot of data which could be irrelevant to the model. With that, the model can learn more from the relevant data hence improving the accuracy of the model. Moreover, the learning rate has been altered to higher which in this model, it proves that it suits higher learning rate of 0.005 than low learning rate of 0.001.</span>

g) In real applications, data-centric strategies are essential to train robust deep learning models. Give two (2) examples of such strategies and discuss how the strategies helps improving the model accuracy. **[5 marks]**

<span style="color:blue">
    One example is to resize the picture to zoom into the important area. By doing it, the model can learn the important part of the data and hence perform better to detect it. Other than that, rotating, flip horizontally and vertically, contrast upon the image data can improve by adjusting more weight of the data, hence more learning is done upon the data and increase the accuracy of the model. </span>

h) Next, implement the two proposed data-centric techniques for the same problem as in the previous question. **[10 marks]**

In [34]:
# h) data centric change at transform

transform_img = transforms.Compose(
     [transforms.ToTensor(),
      transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x),
      transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
      transforms.RandomRotation(degrees=15),
      transforms.RandomHorizontalFlip(),
      transforms.RandomVerticalFlip(),
      transforms.RandomAutocontrast(),
      transforms.CenterCrop(size=224),
      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

 # batch_size
batch_size = 32

# datasets
trainset = torchvision.datasets.FashionMNIST('./data',
     download=True,
     train=True,
     transform=transform_img)
testset = torchvision.datasets.FashionMNIST('./data',
     download=True,
     train=False,
     transform=transform_img)

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
         'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

train_data_size = len(trainloader.dataset)
test_data_size = len(testloader.dataset)

print(train_data_size)
print(test_data_size)

60000
10000


In [36]:
# train again

model_pt = models.googlenet(pretrained=True)
num_ftrs = model_pt.fc.in_features
model_pt.fc = nn.Linear(num_ftrs, 10)
model_pt.dropout = nn.Dropout(p=0.25)

model_pt = model_pt.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_pt.parameters(), lr=0.005, momentum=0.9)

num_epochs = 10
trained_model, history = train_and_validate(model_pt, criterion, optimizer_ft, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.4609, Accuracy: 83.6567%, 
		Validation : Loss : 0.3102, Accuracy: 89.2200%, Time: 360.5399s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.3035, Accuracy: 89.0567%, 
		Validation : Loss : 0.2865, Accuracy: 89.7800%, Time: 364.8429s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.2652, Accuracy: 90.4483%, 
		Validation : Loss : 0.2608, Accuracy: 90.7300%, Time: 366.6201s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.2440, Accuracy: 91.1983%, 
		Validation : Loss : 0.2428, Accuracy: 91.2400%, Time: 365.8434s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.2241, Accuracy: 92.0200%, 
		Validation : Loss : 0.2376, Accuracy: 91.4800%, Time: 362.9558s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.2107, Accuracy: 92.3617%, 
		Validation : Loss : 0.2227, Accuracy: 91.9000%, Time: 365.6022s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.2017, Accuracy: 92.6750%, 
		Validation : Loss : 0.2256, Accuracy: 92.0900%, Time: 366.3211s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.1925,

**QUESTION 2** **[35 marks]**

Firstly, watch this video:

https://drive.google.com/file/d/1bsypahR7I3f_R3DXkfw_tf0BrbCHxE_O/view?usp=sharing

This video shows an example of masked face recognition where the deep learning model is able to detect and classify your face even when wearing a face mask. Using the end-to-end object detection pipeline that you have learned, develop your own masked face recognition such that the model should recognize your face even on face mask while recognize other persons as "others".

Deliverables for this question are:

- the model file. Change the name to <your_name>.pt file (e.g. hasan.pt).
- a short video (~10 secs) containing your face and your friends faces (for inference).

In [37]:
# using yolov5
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 13600, done.
remote: Counting objects: 100% (402/402), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 13600 (delta 292), reused 376 (delta 278), pack-reused 13198
Receiving objects: 100% (13600/13600), 13.27 MiB | 9.98 MiB/s, done.
Resolving deltas: 100% (9356/9356), done.
/content/yolov5
     |████████████████████████████████| 1.6 MB 46.6 MB/s 
     |████████████████████████████████| 145 kB 32.2 MB/s 
     |████████████████████████████████| 178 kB 71.1 MB/s 
     |████████████████████████████████| 1.1 MB 50.2 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 138 kB 71.9 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
Setup complete. Using torch 1.12.1+cu113 (Tesla T4)


In [38]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [39]:
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [40]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="T4GG6fQLiT4CTfzYwU1L")
project = rf.workspace("my-face-zhgsq").project("face-vs-others")
dataset = project.version(3).download("yolov5")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/datasets/Face-vs-Others-3 in yolov5pytorch:: 100%|██████████| 286/286 [00:00<00:00, 2113.71it/s]


In [41]:
!python train.py --img 416 --batch 16 --epochs 150 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/content/datasets/Face-vs-Others-3/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-187-g5ef69ef Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hs

In [ ]:
# name best.pt rename to mardhiah.pt manually at folder runs/train/exp/weights/

In [42]:
!python detect.py --weights runs/train/exp/weights/mardhiah.pt --img 416 --conf 0.1 --source {dataset.location}/test/images

detect: weights=['runs/train/exp/weights/mardhiah.pt'], source=/content/datasets/Face-vs-Others-3/test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-187-g5ef69ef Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/7 /content/datasets/Face-vs-Others-3/test/images/WIN_20221007_15_50_47_Pro_jpg.rf.34ad66b34ec976f1899700fea7fcb112.jpg: 416x416 2 otherss, 8.2ms
image 2/7 /content/datasets/Face-vs-Others-3/test/images/WIN_20221007_15_50_56_Pro_jpg.rf.a9f0c516f9e69eb6368eac73b360418a.jpg: 416x416 2 otherss, 8.2ms
image 3/7 

In [43]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/mardhiah.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>